In [97]:
# Load necessary python libraries

import pandas as pd
import numpy as np
import sqlite3
from difflib import SequenceMatcher # This is for text analysis to find close matches
import time

# Workbook to create master database for analysis / modelling

Initial steps:
1. Determine scope of analysis <br><br>
2. Create a player connection between the Market Values data (sourced from TransferMarkt) and player performance data (sourced from Squawka) <br><br>
3. Start with "Market Values" data as base and add fields / features from other data files

### Step 1: Determine scope of analysis

_Understand the latest market values (as at 2017/18 - i.e. 2017) and compare to the market values from 2016/17_

a. Accordingly, the 'Market Values' for players at 2017 forms the initial basis.

b. Add in a column which has the 2016 market value.

c. Filter the dataset to drop any rows which do not have values for both market value columns (a comparison isn't possible if both values are not available).

In [98]:
# Load market values csv

market_values = pd.read_csv("market_values.csv")

In [99]:
market_values.shape

(16530, 26)

In [100]:
# Create dataframe of 2017 players with market values

mv_df = market_values[market_values['year'] == 2017].copy()

In [101]:
mv_df.shape

(5057, 26)

In [102]:
# Drop players without Market values

mv_df = mv_df[mv_df['marketval'] > 0]
mv_df.shape

(5004, 26)

In [103]:
# Create market values df from prior year (2016)

mv_2016 = market_values[market_values['year'] == 2016].copy() # Create dataframe with 2016 values

# Create market values df from additional prior year (2015)

mv_2015 = market_values[market_values['year'] == 2015].copy()

In [104]:
# Create function to find previous market value (can't simply do a dataframe combination since multiple market
# values exist. Need to select the earliest available - which will be based on "in team since" date)

def prior_year_mv(value):
    if value in mv_2016['pid'].values:
        temp_df = mv_2016[mv_2016['pid'] == value] # Create temporary dataframe just for the identified player
        temp_df = temp_df.sort_values(by = 'teamsince').reset_index() # Re-index so oldest value at 0 index
        prev_val = temp_df.loc[0,'marketval']
    else:
        prev_val = np.NaN # in case player id is not in 2016 dataset
    return prev_val
    
mv_df['marketval_2016'] = mv_df['pid'].apply(prior_year_mv)

In [105]:
# Create function to find country for previous market value (can't simply do a dataframe combination since multiple market
# values exist. Need to select the earliest available - which will be based on "in team since" date)

def prior_year_mv(value):
    if value in mv_2016['pid'].values:
        temp_df = mv_2016[mv_2016['pid'] == value] # Create temporary dataframe just for the identified player
        temp_df = temp_df.sort_values(by = 'teamsince').reset_index() # Re-index so oldest value at 0 index
        prev_val = temp_df.loc[0,'country']
    else:
        prev_val = np.NaN # in case player id is not in 2016 dataset
    return prev_val
    
mv_df['country_2016'] = mv_df['pid'].apply(prior_year_mv)

In [106]:
# Drop players where a market value isn't available for both 2017 and 2016

mv_df = mv_df[mv_df['marketval_2016'] > 0]

In [107]:
mv_df.shape

(3536, 28)

In [108]:
mv_df = mv_df.drop_duplicates()
mv_df.shape

(3177, 28)

### Step 2: Connect with players in statistics data

In [109]:
# Load player stats csv

players = pd.read_csv("player_info.csv")
players.shape

(297116, 17)

In [110]:
# Isolate player Id, name and dob and then remove duplicate player names

players_small = players[['idn','name','dob']]
players_small = players_small.drop_duplicates()
players_small.shape

(8346, 3)

In [111]:
# Create a combined name and dob column in each dataframe

def name_dob(row):
    name_val = row['name'].lower().replace(" ","_")
    dob_val = str(row['dob'])
    combo = name_val + "_" + dob_val
    return combo

for df in [mv_df,players_small]:
    df['name_dob'] = df.apply(name_dob, axis = 1)

In [112]:
mv_df.shape

(3177, 29)

In [113]:
# Check how many exact matches there are with the Squawka list

def match_name(value):
    temp_df = players_small[players_small['name_dob'] == value]
    shape_val = temp_df.shape[0]
    return shape_val

mv_df['check'] = mv_df['name_dob'].apply(match_name)

In [114]:
mv_df['check'].value_counts()

1    2843
0     333
2       1
Name: check, dtype: int64

In [115]:
# Review dataframe of player with more than 1 match

mv_df[mv_df['check'] == 2]

,teamid,number,name,pid,main_pos,position,dob,age,nat1,nat2,...,season,teamurl,year,current_team,current_teamid,country,marketval_2016,country_2016,name_dob,check
7936,610.0,21.0,Frenkie de Jong,326330.0,midfielder,Central Midfield,1997-05-12,20.0,Netherlands,NaN,...,17/18,ajax-amsterdam,2017.0,Ajax Amsterdam,610.0,Netherlands,600000.0,Netherlands,frenkie_de_jong_1997-05-12,2


In [116]:
# Create dataframe of unmatched players
mv_df_extra = mv_df[mv_df['check'] == 0].copy()

##### Attempt to find players that are not exact matches but may be due to close match (e.g. minor wording difference)

In [117]:
mv_df.head(5)

,teamid,number,name,pid,main_pos,position,dob,age,nat1,nat2,...,season,teamurl,year,current_team,current_teamid,country,marketval_2016,country_2016,name_dob,check
594,8054.0,1.0,Andrew Redmayne,51975.0,keeper,Keeper,1989-01-13,28.0,Australia,NaN,...,17/18,sydney-fc,2017.0,Sydney FC,8054.0,Australia,300000.0,Australia,andrew_redmayne_1989-01-13,1
596,8054.0,5.0,Jordy Buijs,31111.0,defender,Centre-Back,1988-12-28,28.0,Netherlands,NaN,...,17/18,sydney-fc,2017.0,Sydney FC,8054.0,Australia,800000.0,Australia,jordy_buijs_1988-12-28,1
597,8054.0,4.0,Alex Wilkinson,43128.0,defender,Centre-Back,1984-08-13,33.0,Australia,NaN,...,17/18,sydney-fc,2017.0,Sydney FC,8054.0,Australia,650000.0,Australia,alex_wilkinson_1984-08-13,1
598,8054.0,22.0,Sebastian Ryall,58120.0,defender,Centre-Back,1989-07-18,28.0,Australia,NaN,...,17/18,sydney-fc,2017.0,Sydney FC,8054.0,Australia,350000.0,Australia,sebastian_ryall_1989-07-18,1
601,8054.0,7.0,Michael Zullo,55444.0,defender,Left-Back,1988-11-09,28.0,Australia,Italy,...,17/18,sydney-fc,2017.0,Sydney FC,8054.0,Australia,350000.0,Australia,michael_zullo_1988-11-09,0


In [118]:
# Check if any players have multiple player ids

mv_df_duplicate_check = mv_df[['pid','name_dob']]
mv_df_duplicate_check = mv_df_duplicate_check.drop_duplicates()

mv_df_duplicate_check['count'] = mv_df_duplicate_check.groupby('name_dob')['name_dob'].transform('count')
mv_df_duplicate_check[mv_df_duplicate_check['count'] != 1]

,pid,name_dob,count


In [119]:
# Check if any players have multiple teams in 2017
mv_df_team_check = mv_df[['pid','current_team']]
mv_df_team_check = mv_df_team_check.drop_duplicates()

mv_df_team_check['count'] = mv_df_team_check.groupby('pid')['pid'].transform('count')
mv_df_team_check[mv_df_team_check['count'] != 1]

,pid,current_team,count


In [120]:
# Since I'll be trying to match on shirt numbers at the current club, 
# I need to match club names across the databases

# Create TransferMarkt club name list

mv_df_teamlist = mv_df[['current_teamid','current_team','country']].copy()
mv_df_teamlist = mv_df_teamlist.drop_duplicates()

# There is no country field in the Squawka data so code to double check no team names are the same across leagues

mv_df_teamlist['count'] = mv_df_teamlist.groupby('current_team')['current_team'].transform('count')
mv_df_teamlist[mv_df_teamlist['count'] != 1]

,current_teamid,current_team,country,count


In [121]:
# Create Squawka club name list

players_teamlist = players[['team_name','team_id']].copy().drop_duplicates()

In [122]:
# Check how many team name matches

def match_team(value):
    temp_df = players_teamlist[players_teamlist['team_name'] == value]
    shape_val = temp_df.shape[0]
    return shape_val

mv_df_teamlist['check'] = mv_df_teamlist['current_team'].apply(match_team)

In [123]:
mv_df_teamlist['check'].value_counts()

0    143
1     23
Name: check, dtype: int64

In [124]:
# Create dataframe of no matches
mv_df_teamlist = mv_df_teamlist[mv_df_teamlist['check'] == 0].copy()

In [125]:
# Create function to compare the TransferMarkt team without a match to each of the teams in the
# Squawka listing. This is done by creating a temporary dataframe which scores how close the TransferMarkt 
# team names match to each of the Squawka team names. It then chooses the team with the highest match value.

def closest_match_team(value):
    string2 = value
    
    def match_rating(string1):
        m = SequenceMatcher(None, string1, string2)
        score = m.ratio()
        return score
    
    players_teamlist['match_rate'] = players_teamlist['team_name'].apply(match_rating)
    
    temp_df = players_teamlist.sort_values(by = 'match_rate', ascending=False).reset_index() # Re-index so oldest value at 0 index
    
    top_match_val = temp_df.loc[0,'team_name']
    
    print("{} done".format(value))
    
    return top_match_val

In [126]:
mv_df_teamlist['closest_match'] = mv_df_teamlist['current_team'].apply(closest_match_team)

Melbourne City FC done
Newcastle United Jets done
Chelsea FC done
Manchester City done
Manchester United done
Arsenal FC done
Tottenham Hotspur done
Liverpool FC done
Everton FC done
Southampton FC done
Leicester City done
West Ham United done
Crystal Palace done
Stoke City done
Watford FC done
Swansea City done
West Bromwich Albion done
Newcastle United done
AFC Bournemouth done
Brighton & Hove Albion done
Burnley FC done
Huddersfield Town done
Los Angeles Galaxy done
Atlanta United FC done
Portland Timbers done
Chicago Fire done
Seattle Sounders FC done
Orlando City SC done
Vancouver Whitecaps done
New York City FC done
San Jose Earthquakes done
New England Revolution done
New York Red Bulls done
Montreal Impact done
Columbus Crew SC done
Real Salt Lake City done
Colorado Rapids done
Houston Dynamo done
Sporting Kansas City done
Minnesota United FC done
Besiktas JK done
Galatasaray SK done
Fenerbahce SK done
Medipol Basaksehir done
Trabzonspor done
Osmanlispor FK done
Atiker Konyaspo

In [127]:
# Create a function to then insert the matching score into the dataframe

def reverse_match_rating_team(row):
    
    string1 = row['current_team']
    string2 = row['closest_match']
    
    m = SequenceMatcher(None, string1, string2)
    score = m.ratio()
    return score

In [128]:
# Calculate to insert the score of the sequence matcher

mv_df_teamlist['match_score'] = mv_df_teamlist.apply(reverse_match_rating_team, axis = 1)

In [129]:
# Manually review list for whether closest match is correct.
mv_df_teamlist.sort_values(by = 'match_score', ascending = False, inplace = True)
mv_df_teamlist.iloc[130:155,:]

,current_teamid,current_team,country,count,check,closest_match,match_score
7975,383.0,PSV Eindhoven,Netherlands,1,0,St Etienne,0.521739
16139,18.0,Borussia Mönchengladbach,Germany,1,0,M'gladbach,0.514286
8000,1090.0,AZ Alkmaar,Netherlands,1,0,Alanyaspor,0.500000
8121,317.0,FC Twente Enschede,Netherlands,1,0,Twente,0.500000
10063,273.0,Stade Rennais FC,France,1,0,Sydney FC,0.480000
6484,1467.0,Göztepe,Turkey,1,0,Gaziantepspor,0.476190
6624,19771.0,Teleset Mobilya Akhisarspor,Turkey,1,0,Akhisar Belediyespor,0.468085
2518,148.0,Tottenham Hotspur,England,1,0,Konyaspor,0.461538
9933,1041.0,Olympique Lyon,France,1,0,Lyon,0.444444
9826,583.0,Paris Saint-Germain,France,1,0,Brisbane,0.444444


In [130]:
# Create list of indexes which didn't match

team_mismatch = [2518,2787,2815,3639,4172,6484,6604,7919,7975,8000,9826,10063,
                11956,12002,14403,14320,14623]

# Insert column in dataframe that manually records the correct matches

def insert_correct(value):
    if value in team_mismatch:
        return 1
    else:
        return 0

mv_df_teamlist['manual_check'] = mv_df_teamlist.index.map(lambda x: 0 if x in team_mismatch else 1)

In [131]:
missing_teams = mv_df_teamlist[mv_df_teamlist['manual_check'] == 0].copy()
missing_teams

,current_teamid,current_team,country,count,check,closest_match,match_score,manual_check
14403,416.0,Torino FC,Italy,1,0,Toronto FC,0.842105,0
2815,762.0,Newcastle United,England,1,0,Newcastle Jets,0.733333,0
14623,862.0,Chievo Verona,Italy,1,0,Verona,0.631579,0
3639,51663.0,Atlanta United FC,United States,1,0,Adelaide United,0.625000,0
14320,46.0,Inter Milan,Italy,1,0,Milan,0.625000,0
12002,714.0,RCD Espanyol Barcelona,Spain,1,0,Barcelona,0.580645,0
6604,19789.0,Evkur Yeni Malatyaspor,Turkey,1,0,Antalyaspor,0.545455,0
7919,610.0,Ajax Amsterdam,Netherlands,1,0,Sparta Rotterdam,0.533333,0
4172,56089.0,Minnesota United FC,United States,1,0,Adelaide United,0.529412,0
7975,383.0,PSV Eindhoven,Netherlands,1,0,St Etienne,0.521739,0


In [132]:
list_names = players_teamlist[['team_name']].sort_values(by = "team_name")

In [133]:
# Some teams won't have player stats since they were only promoted to their respective leagues in 2017/18

players_teamlist_names = ['Torino','Newcastle','Chievo',np.NaN,'Inter','Espanyol',np.NaN,'Ajax',np.NaN,
                         'PSV','AZ','Rennes',np.NaN,'Spurs','PSG','WBA','Betis']

# Append team names to dataframe

missing_teams['teamname_manual'] = players_teamlist_names

In [134]:
# Combine team name dataframes
def man_team_name(value):
    if value in missing_teams.index:
        teamname = missing_teams.loc[value,'teamname_manual']
    else:
        teamname = np.NaN
    
    return teamname


mv_df_teamlist['index_val'] = mv_df_teamlist.index
mv_df_teamlist['manual_check_2'] = mv_df_teamlist['index_val'].apply(man_team_name)

In [135]:
mv_df_master_teams = mv_df[['teamid','current_team']].copy()
mv_df_master_teams = mv_df_master_teams.drop_duplicates()
mv_df_master_teams['index_val'] = mv_df_master_teams.index # Create a column for the index (to be used for a function later)

In [136]:
mv_df_teamlist

,current_teamid,current_team,country,count,check,closest_match,match_score,manual_check,index_val,manual_check_2
673,25580.0,Melbourne City FC,Australia,1,0,Melbourne City,0.903226,1,673,NaN
6220,141.0,Galatasaray SK,Turkey,1,0,Galatasaray,0.880000,1,6220,NaN
6359,2944.0,Osmanlispor FK,Turkey,1,0,Osmanlispor,0.880000,1,6359,NaN
8054,306.0,SC Heerenveen,Netherlands,1,0,Heerenveen,0.869565,1,8054,NaN
11931,472.0,UD Las Palmas,Spain,1,0,Las Palmas,0.869565,1,11931,NaN
11817,1050.0,Villarreal CF,Spain,1,0,Villarreal,0.869565,1,11817,NaN
14485,1038.0,UC Sampdoria,Italy,1,0,Sampdoria,0.857143,1,14485,NaN
8101,202.0,FC Groningen,Netherlands,1,0,Groningen,0.857143,1,8101,NaN
2542,31.0,Liverpool FC,England,1,0,Liverpool,0.857143,1,2542,NaN
11745,131.0,FC Barcelona,Spain,1,0,Barcelona,0.857143,1,11745,NaN


In [137]:
# Create a mapping of team names in the squawka list

def insert_squawka_teamname(row):
    if row['index_val'] not in mv_df_teamlist.index: # This means that the team name has an exact match in the Squawka list
        teamname = row['current_team']
    
    elif mv_df_teamlist.loc[row['index_val'],'manual_check'] == 1:
        teamname = mv_df_teamlist.loc[row['index_val'],'closest_match']
    
    
    elif mv_df_teamlist.loc[row['index_val'],'manual_check_2'] != np.NaN:
        teamname = mv_df_teamlist.loc[row['index_val'],'manual_check_2']

    elif mv_df_teamlist.loc[row['index_val'],'manual_check_2'] == np.NaN:
        teamname = np.NaN
        
    return teamname

mv_df_master_teams['squawka_name'] = mv_df_master_teams.apply(insert_squawka_teamname, axis = 1)

In [138]:
mv_df_master_teams.shape

(166, 4)

In [139]:
mv_df_master_teams.head(10)

,teamid,current_team,index_val,squawka_name
594,8054.0,Sydney FC,594,Sydney FC
619,36100.0,Western Sydney Wanderers,619,Western Sydney Wanderers
648,7831.0,Brisbane Roar,648,Brisbane Roar
673,25580.0,Melbourne City FC,673,Melbourne City
699,8445.0,Wellington Phoenix,699,Wellington Phoenix
724,7006.0,Melbourne Victory,724,Melbourne Victory
746,970.0,Perth Glory,746,Perth Glory
767,6954.0,Central Coast Mariners,767,Central Coast Mariners
790,875.0,Adelaide United,790,Adelaide United
811,5551.0,Newcastle United Jets,811,Newcastle Jets


In [140]:
players_teamlist.columns = ['squawka_name','team_id','match_rate']
players_teamlist.drop('match_rate', axis = 1, inplace=True)
players_teamlist.head(2)

,squawka_name,team_id
0,Swansea,46.0
18,Leicester,315.0


In [141]:
# Master dataframe that maps the team name to the squawka team name

mv_df_master_teams = pd.merge(mv_df_master_teams,players_teamlist,how='left',on='squawka_name')
mv_df_master_teams.drop('index_val', axis = 1, inplace = True)

In [142]:
mv_df_master_teams.head(20)

,teamid,current_team,squawka_name,team_id
0,8054.0,Sydney FC,Sydney FC,332.0
1,36100.0,Western Sydney Wanderers,Western Sydney Wanderers,334.0
2,7831.0,Brisbane Roar,Brisbane Roar,326.0
3,25580.0,Melbourne City FC,Melbourne City,328.0
4,8445.0,Wellington Phoenix,Wellington Phoenix,333.0
5,7006.0,Melbourne Victory,Melbourne Victory,329.0
6,970.0,Perth Glory,Perth Glory,331.0
7,6954.0,Central Coast Mariners,Central Coast Mariners,327.0
8,875.0,Adelaide United,Adelaide United,325.0
9,5551.0,Newcastle United Jets,Newcastle Jets,330.0


In [143]:
# Convert dataframes to CSV

start_time=time.time()
start_time_stamp = str(start_time).replace(".","")
start_time_stamp

team_file_name = "team_" + start_time_stamp + ".csv"

mv_df_master_teams.to_csv(team_file_name, encoding="utf-8", index = False)

In [144]:
players.head()

,index,age,bmi,country,dob,first_name,height,last_name,name,position,shirt_num,state,surname,team_id,team_name,weight,idn
0,0,27.0,21.2,Wales,1989-02-07,Neil,175.0,Taylor,Neil Taylor,Defender,0.0,playing,Neil Taylor,46.0,Swansea,65.0,817.0
1,1,32.0,23.0,Wales,1984-08-23,Ashley,183.0,Williams,Ashley Williams,Defender,0.0,playing,Ashley Williams,46.0,Swansea,77.0,810.0
2,2,25.0,22.2,England,1991-05-22,Kyle,185.0,Bartley,Kyle Bartley,Defender,0.0,bench,Kyle Bartley,46.0,Swansea,76.0,382.0
3,3,31.0,23.0,Poland,1985-04-18,Lukasz,190.0,Fabianski,Lukasz Fabianski,Goalkeeper,1.0,playing,Fabianski,46.0,Swansea,83.0,375.0
4,4,28.0,22.6,Korea Republic,1989-01-24,Sung-yueng,187.0,Ki,Ki Sung-yueng,Midfielder,4.0,playing,Ki,46.0,Swansea,79.0,3691.0


In [145]:
# Create function to compare the TransferMarkt name_dob without a match to each of the players in the
# Squawka listing. This is done by creating a temporary dataframe which scores how close the TransferMarkt 
# players match to each of the Squawka player names. It then chooses the player with the highest match value.

def closest_match(value):
    string2 = value
    
    def match_rating(string1):
        m = SequenceMatcher(None, string1, string2)
        score = m.ratio()
        return score
    
    players_small['match_rate'] = players_small['name_dob'].apply(match_rating)
    
    temp_df = players_small.sort_values(by = 'match_rate', ascending=False).reset_index() # Re-index so oldest value at 0 index
    
    top_match_val = temp_df.loc[0,'name_dob']
    
    print("{} done".format(value))
    
    return top_match_val

In [146]:
mv_df_extra['closest_match'] = mv_df_extra['name_dob'].apply(closest_match)

michael_zullo_1988-11-09 done
eric_bauthéac_1987-08-24 done
nicholas_fitzgerald_1992-02-13 done
hernán_espíndola_1994-10-19 done
dylan_fox_1994-01-01 done
thomas_doyle_1992-06-30 done
matt_acton_1992-06-03 done
mitch_austin_1991-04-03 done
asdrúbal_padrón_1991-03-13 done
ben_warland_1997-01-01 done
benjamin_garuccio_1995-06-15 done
babá_1988-01-05 done
nicholas_cowburn_1995-03-07 done
ben_kantarovski_1992-01-20 done
matej_delac_1992-08-20 done
wallace_oliveira_1994-05-01 done
danny_drinkwater_1990-03-05 done
charly_musonda_jr._1996-10-15 done
santi_cazorla_1984-12-13 done
georges-kevin_n'koudou_1995-02-13 done
Érik_lamela_1992-03-04 done
Ádám_bogdán_1987-09-27 done
joe_gomez_1997-05-23 done
pierre-emile_höjbjerg_1995-08-05 done
onyinye_ndidi_1996-12-16 done
julián_speroni_1979-05-18 done
chung-yong_lee_1988-07-02 done
mame_diouf_1987-12-16 done
younès_kaboul_1986-01-04 done
molla_wague_1991-02-21 done
brice_dja_djedjé_1990-12-23 done
Étienne_capoue_1988-07-11 done
mike_van_der_hoorn_19

héctor_hernández_1991-05-23 done
bojan_krkic_1990-08-28 done
rubén_sobrino_1992-02-16 done
jon_ander_serantes_1989-01-04 done
iván_cuéllar_1984-05-25 done
javi_eraso_1990-03-22 done
josé_naranjo_1994-07-28 done
omar_1988-01-26 done
mamadou_koné_1991-12-25 done
fayçal_fajr_1988-08-01 done
dani_pacheco_1991-01-05 done
alex_alegría_1992-10-14 done
pepe_reina_1982-08-31 done
rafael_1990-05-20 done
camilo_zúñiga_1985-12-14 done
cristián_zapata_1986-09-30 done
jherson_vergara_1994-05-26 done
konstantinos_manolas_1991-06-14 done
emerson_1994-08-03 done
grégoire_defrel_1991-06-17 done
assane_gnoukouri_1996-09-28 done
maurício_1988-09-20 done
kevin_bonifazi_1996-05-19 done
nicolás_burdisso_1981-04-12 done
Álex_berenguer_1995-07-04 done
rafael_tolói_1990-10-10 done
papu_gómez_1988-02-15 done
gianmarco_ferrari_1992-05-15 done
leonardo_capezzi_1995-03-29 done
Édgar_barreto_1984-07-15 done
loïck_landre_1992-05-05 done
rodney_strasser_1990-03-30 done
isaac_cofie_1991-09-20 done
riad_bajić_1994-05-06

In [147]:
# Create a function to then insert the matching score into the dataframe

def reverse_match_rating(row):
    
    string1 = row['name_dob']
    string2 = row['closest_match']
    
    m = SequenceMatcher(None, string1, string2)
    score = m.ratio()
    return score

In [148]:
# Calculate to insert the score of the sequence matcher

mv_df_extra['match_score'] = mv_df_extra.apply(reverse_match_rating, axis = 1)

In [149]:
# Sort dataframe by matching score

mv_df_extra = mv_df_extra.sort_values(by = 'match_score', ascending = False)

In [150]:
mv_df_extra.head()

,teamid,number,name,pid,main_pos,position,dob,age,nat1,nat2,...,year,current_team,current_teamid,country,marketval_2016,country_2016,name_dob,check,closest_match,match_score
14582,1025.0,35.0,Vasilios Torosidis,24977.0,defender,Right-Back,1985-06-10,32.0,Greece,NaN,...,2017.0,Bologna FC 1909,1025.0,Italy,3000000.0,Italy,vasilios_torosidis_1985-06-10,0,vasilis_torosidis_1985-06-10,0.982456
10128,855.0,5.0,Mustapha Diallo,41063.0,midfielder,Defensive Midfield,1986-05-14,31.0,Senegal,NaN,...,2017.0,EA Guingamp,855.0,France,1500000.0,France,mustapha_diallo_1986-05-14,0,moustapha_diallo_1986-05-14,0.981132
2618,180.0,23.0,Pierre-Emile Höjbjerg,167799.0,midfielder,Central Midfield,1995-08-05,22.0,Denmark,France,...,2017.0,Southampton FC,180.0,England,6000000.0,England,pierre-emile_höjbjerg_1995-08-05,0,pierre-emile_højbjerg_1995-08-05,0.969697
10032,618.0,23.0,Alexander Söderlund,87479.0,attacker,Centre-Forward,1987-08-03,30.0,Norway,NaN,...,2017.0,AS Saint-Étienne,618.0,France,2000000.0,France,alexander_söderlund_1987-08-03,0,alexander_søderlund_1987-08-03,0.967742
8218,1269.0,33.0,Athanasios Karagounis,105811.0,attacker,Right Wing,1991-09-25,25.0,Greece,NaN,...,2017.0,PEC Zwolle,1269.0,Netherlands,500000.0,Netherlands,athanasios_karagounis_1991-09-25,0,thanasis_karagounis_1991-09-25,0.967742


In [151]:
# Create dataframe of 2017 players with a column that combines team_name and shirt_number
# This is to be used to then look up the name_dob for that team_shirt combination and insert into the above df
# This will act as a double-check that the correct player has been identified and also locate correct player names

# Step a: Load the 'stats' CSV which has the year filter. Only keep idn (player id), team_id & start_year (2017)
# Step b: Create a new column in a new players DF which combines team_id and pid
# Step c: Retrieve the shirt_num, player_name dob from the players dataframe (based on the team_id & idn combo)
# Step d: Create a new column in this new DF which combines team id and shirt id
# Step e: Retrieve the matched squawka team_id from the mv_df_master_teams & insert into mv_df_extra
# Step f: Create a new column in mv_df_extra that combines the shirt num and this team id
# Step g: Retrieve the matched name_dob from step c DF based on the shirt_num and team id match

In [152]:
# Step a: Load the 'stats' CSV which has the year filter. Only keep idn (player id), team_id and start_year

match_stats = pd.read_csv("player_stats.csv")
match_stats_2017 = match_stats[match_stats['start_year'] == 2017][['idn','team_id','start_year']]
match_stats_2017 = match_stats_2017.drop_duplicates()
match_stats_2017['team_id_idn'] = match_stats_2017['team_id'].astype(str) + "_" + match_stats_2017['idn'].astype(str) 
match_stats_2017.head(2)

,idn,team_id,start_year,team_id_idn
221081,92.0,44.0,2017,44.0_92.0
221082,275.0,48.0,2017,48.0_275.0


In [153]:
extra_player_stats_db = '../sql/stats_extra_games.db.sqlite'
conn_6 = sqlite3.connect(extra_player_stats_db) 
c = conn_6.cursor()

# This is my SQL query to select everything:
table_6 = 'select * from playertable'

# Use `read_sql` from the `pandas` SQL library and set it equal to a DF object:
players_master_2017 = pd.read_sql(table_6, con = conn_6)

In [154]:
# Step b: Create a new column in a new players DF which combines team_id and pid

players_master = players_master_2017[['idn','name','dob','shirt_num','team_id']]
players_master = players_master.drop_duplicates()
players_master['team_id_idn'] = players_master['team_id'].astype(str) + "_" + players_master['idn'].astype(str)
players_master.head(2)

# Create a combined name and dob column in the dataframe

def name_dob(row):
    name_val = row['name'].lower().replace(" ","_")
    dob_val = str(row['dob'])
    combo = name_val + "_" + dob_val
    return combo

players_master['name_dob'] = players_master.apply(name_dob, axis = 1)

players_master.head(2)

,idn,name,dob,shirt_num,team_id,team_id_idn,name_dob
0,92.0,Jack Butland,1993-03-10,1.0,44.0,44.0_92.0,jack_butland_1993-03-10
1,5429.0,Erik Pieters,1988-08-07,3.0,44.0,44.0_5429.0,erik_pieters_1988-08-07


In [155]:
# Step c: Retrieve the shirt_num, player_name dob from the players dataframe (based on the team_id & idn combo)

match_stats_2017 = pd.merge(match_stats_2017,players_master[['team_id_idn','shirt_num','name_dob']],how='left',on='team_id_idn')
match_stats_2017.head(5)

,idn,team_id,start_year,team_id_idn,shirt_num,name_dob
0,92.0,44.0,2017,44.0_92.0,1.0,jack_butland_1993-03-10
1,275.0,48.0,2017,48.0_275.0,14.0,james_mcclean_1989-04-22
2,540.0,44.0,2017,44.0_540.0,4.0,joe_allen_1990-03-14
3,546.0,48.0,2017,48.0_546.0,18.0,gareth_barry_1981-02-23
4,572.0,44.0,2017,44.0_572.0,24.0,darren_fletcher_1984-02-01


In [156]:
match_stats_2017.shape

(1963, 6)

In [157]:
# Check if any players have multiple shirt numbers

match_stats_2017['count'] = match_stats_2017.groupby('team_id_idn')['team_id_idn'].transform('count')
match_stats_2017[match_stats_2017['count'] != 1].shape

(60, 7)

In [158]:
match_stats_2017[(match_stats_2017['shirt_num'] == 0) & (match_stats_2017['count'] > 1)].shape

(27, 7)

In [159]:
# Drop any players where multiple shirt number is 0

match_stats_2017 = match_stats_2017.drop(match_stats_2017[(match_stats_2017['shirt_num'] == 0) & (match_stats_2017['count'] > 1)].index)

In [160]:
match_stats_2017.shape

(1936, 7)

In [161]:
# Check if any players have multiple shirt numbers

match_stats_2017['count'] = match_stats_2017.groupby('team_id_idn')['team_id_idn'].transform('count')
match_stats_2017[match_stats_2017['count'] != 1]

,idn,team_id,start_year,team_id_idn,shirt_num,name_dob,count
208,20573.0,323.0,2017,323.0_20573.0,19.0,richarlison_1997-05-10,2
209,20573.0,323.0,2017,323.0_20573.0,11.0,richarlison_1997-05-10,2
383,1307.0,133.0,2017,133.0_1307.0,4.0,milan_bisevac_1983-08-31,2
384,1307.0,133.0,2017,133.0_1307.0,6.0,milan_bisevac_1983-08-31,2
470,1210.0,67.0,2017,67.0_1210.0,18.0,rémy_cabella_1990-03-08,2
471,1210.0,67.0,2017,67.0_1210.0,10.0,rémy_cabella_1990-03-08,2


In [162]:
# Step d: Create a new column in this new DF which combines team id and shirt id

match_stats_2017['team_id_shirt'] = match_stats_2017['team_id'].astype(str) + "_" + match_stats_2017['shirt_num'].astype(str)

In [163]:
# Check team-shirt duplicates where shirt number not 0

match_stats_2017['count_2'] = match_stats_2017.groupby('team_id_shirt')['team_id_shirt'].transform('count')
match_stats_2017[(match_stats_2017['count_2'] != 1) & (match_stats_2017['shirt_num'] != 0)]

,idn,team_id,start_year,team_id_idn,shirt_num,name_dob,count,team_id_shirt,count_2
596,21342.0,56.0,2017,56.0_21342.0,29.0,rominigue_kouame_1996-12-17,1,56.0_29.0,2
631,12253.0,53.0,2017,53.0_12253.0,7.0,otávio_1994-05-04,1,53.0_7.0,2
632,12259.0,53.0,2017,53.0_12259.0,7.0,malcom_1997-02-26,1,53.0_7.0,2
658,17039.0,56.0,2017,56.0_17039.0,29.0,thiago_maia_1997-03-23,1,56.0_29.0,2
881,1942.0,89.0,2017,89.0_1942.0,10.0,éver_banega_1988-06-29,1,89.0_10.0,2
890,5715.0,525.0,2017,525.0_5715.0,12.0,míchel_macedo_1990-02-15,1,525.0_12.0,2
891,6394.0,89.0,2017,89.0_6394.0,10.0,ganso_1989-10-12,1,89.0_10.0,2
898,20623.0,525.0,2017,525.0_20623.0,12.0,jonathan_calleri_1993-09-23,1,525.0_12.0,2
1029,12503.0,102.0,2017,102.0_12503.0,5.0,walace_1995-04-04,1,102.0_5.0,2
1214,7530.0,95.0,2017,95.0_7530.0,19.0,mahmoud_dahoud_1996-01-01,1,95.0_19.0,2


In [165]:
# Step e: Retrieve the matched squawka team_id from the mv_df_master_teams & insert into mv_df_extra

mv_df_master_teams.columns = ['teamid','TM_team','squawka_name','squawka_teamid']

mv_df_extra = pd.merge(mv_df_extra,mv_df_master_teams[['teamid','squawka_teamid']],how='left',on='teamid')
mv_df_extra.head(5)

,teamid,number,name,pid,main_pos,position,dob,age,nat1,nat2,...,current_team,current_teamid,country,marketval_2016,country_2016,name_dob,check,closest_match,match_score,squawka_teamid
0,1025.0,35.0,Vasilios Torosidis,24977.0,defender,Right-Back,1985-06-10,32.0,Greece,NaN,...,Bologna FC 1909,1025.0,Italy,3000000.0,Italy,vasilios_torosidis_1985-06-10,0,vasilis_torosidis_1985-06-10,0.982456,110.0
1,855.0,5.0,Mustapha Diallo,41063.0,midfielder,Defensive Midfield,1986-05-14,31.0,Senegal,NaN,...,EA Guingamp,855.0,France,1500000.0,France,mustapha_diallo_1986-05-14,0,moustapha_diallo_1986-05-14,0.981132,144.0
2,180.0,23.0,Pierre-Emile Höjbjerg,167799.0,midfielder,Central Midfield,1995-08-05,22.0,Denmark,France,...,Southampton FC,180.0,England,6000000.0,England,pierre-emile_höjbjerg_1995-08-05,0,pierre-emile_højbjerg_1995-08-05,0.969697,43.0
3,618.0,23.0,Alexander Söderlund,87479.0,attacker,Centre-Forward,1987-08-03,30.0,Norway,NaN,...,AS Saint-Étienne,618.0,France,2000000.0,France,alexander_söderlund_1987-08-03,0,alexander_søderlund_1987-08-03,0.967742,67.0
4,1269.0,33.0,Athanasios Karagounis,105811.0,attacker,Right Wing,1991-09-25,25.0,Greece,NaN,...,PEC Zwolle,1269.0,Netherlands,500000.0,Netherlands,athanasios_karagounis_1991-09-25,0,thanasis_karagounis_1991-09-25,0.967742,243.0


In [166]:
# Step f: Create a new column in mv_df_extra that combines the shirt num and this team id

mv_df_extra['team_id_shirt'] = mv_df_extra['squawka_teamid'].astype(str) + "_" + mv_df_extra['number'].astype(str)
mv_df_extra.head(2)

,teamid,number,name,pid,main_pos,position,dob,age,nat1,nat2,...,current_teamid,country,marketval_2016,country_2016,name_dob,check,closest_match,match_score,squawka_teamid,team_id_shirt
0,1025.0,35.0,Vasilios Torosidis,24977.0,defender,Right-Back,1985-06-10,32.0,Greece,NaN,...,1025.0,Italy,3000000.0,Italy,vasilios_torosidis_1985-06-10,0,vasilis_torosidis_1985-06-10,0.982456,110.0,110.0_35.0
1,855.0,5.0,Mustapha Diallo,41063.0,midfielder,Defensive Midfield,1986-05-14,31.0,Senegal,NaN,...,855.0,France,1500000.0,France,mustapha_diallo_1986-05-14,0,moustapha_diallo_1986-05-14,0.981132,144.0,144.0_5.0


In [167]:
# Step g: Retrieve the matched name_dob from step c DF based on the shirt_num and team id match

mv_df_extra = pd.merge(mv_df_extra,match_stats_2017[['team_id_shirt','name_dob']],how='left',on='team_id_shirt')
mv_df_extra.head(5)

,teamid,number,name,pid,main_pos,position,dob,age,nat1,nat2,...,country,marketval_2016,country_2016,name_dob_x,check,closest_match,match_score,squawka_teamid,team_id_shirt,name_dob_y
0,1025.0,35.0,Vasilios Torosidis,24977.0,defender,Right-Back,1985-06-10,32.0,Greece,NaN,...,Italy,3000000.0,Italy,vasilios_torosidis_1985-06-10,0,vasilis_torosidis_1985-06-10,0.982456,110.0,110.0_35.0,NaN
1,855.0,5.0,Mustapha Diallo,41063.0,midfielder,Defensive Midfield,1986-05-14,31.0,Senegal,NaN,...,France,1500000.0,France,mustapha_diallo_1986-05-14,0,moustapha_diallo_1986-05-14,0.981132,144.0,144.0_5.0,moustapha_diallo_1986-05-14
2,180.0,23.0,Pierre-Emile Höjbjerg,167799.0,midfielder,Central Midfield,1995-08-05,22.0,Denmark,France,...,England,6000000.0,England,pierre-emile_höjbjerg_1995-08-05,0,pierre-emile_højbjerg_1995-08-05,0.969697,43.0,43.0_23.0,NaN
3,618.0,23.0,Alexander Söderlund,87479.0,attacker,Centre-Forward,1987-08-03,30.0,Norway,NaN,...,France,2000000.0,France,alexander_söderlund_1987-08-03,0,alexander_søderlund_1987-08-03,0.967742,67.0,67.0_23.0,hernani_1994-03-27
4,1269.0,33.0,Athanasios Karagounis,105811.0,attacker,Right Wing,1991-09-25,25.0,Greece,NaN,...,Netherlands,500000.0,Netherlands,athanasios_karagounis_1991-09-25,0,thanasis_karagounis_1991-09-25,0.967742,243.0,243.0_33.0,NaN


In [168]:
players_small.head(2)

,idn,name,dob,name_dob,match_rate
0,817.0,Neil Taylor,1989-02-07,neil_taylor_1989-02-07,0.416667
1,810.0,Ashley Williams,1984-08-23,ashley_williams_1984-08-23,0.346154


In [169]:
# Insert appearances from 2016 into mv_name_review

career_summary = pd.read_csv("career_summary.csv")

mv_df_extra = pd.merge(mv_df_extra,career_summary[['pid','apps']],how='left',on='pid')

In [170]:
mv_name_review = mv_df_extra[['pid','number','apps','current_team','name','name_dob_x','closest_match','match_score','team_id_shirt',
                             'name_dob_y']].copy()
mv_name_review[mv_name_review['match_score'] > .78].shape

(284, 10)

I manually reviewed the matches found by the Sequence Matcher and found the following:

* All of the results with a score above 0.78 are correct (284 players), except for the following Transfer Markt ids:
    * 103916 (map to Squawka player ID: 5572)
    * 76285 (map to Squawka player ID: 21071)
   

* For the results with a score below 0.78 (50 players), only the following were correct:
    * Transfer Markt player IDs: 37941, 113116, 221986, 263843, 59938, 60444, 61812, 5782, 20005
    * ID 58884 needs to be mapped to Squawka player ID: 3593

In [171]:
mv_name_review.head(2)

,pid,number,apps,current_team,name,name_dob_x,closest_match,match_score,team_id_shirt,name_dob_y
0,24977,35.0,373.0,Bologna FC 1909,Vasilios Torosidis,vasilios_torosidis_1985-06-10,vasilis_torosidis_1985-06-10,0.982456,110.0_35.0,NaN
1,41063,5.0,239.0,EA Guingamp,Mustapha Diallo,mustapha_diallo_1986-05-14,moustapha_diallo_1986-05-14,0.981132,144.0_5.0,moustapha_diallo_1986-05-14


In [172]:
# Create lists and dictionaries relating to the above notes

correct_ids_under_pt78 = [37941, 113116, 221986, 263843, 59938, 60444, 61812, 5782, 20005]

correct_ids_under_pt78 = [float(x) for x in correct_ids_under_pt78]

mv_name_review['pid'] = mv_name_review['pid'].apply(pd.to_numeric, errors = 'coerce')

remapping_ids = {103916.0: 5572.0, 76285.0: 21071.0, 58884.0: 3593.0}

In [173]:
found_key = 103916
players_small[players_small['idn'] == remapping_ids[found_key]]['name_dob']

96788    rafael_cabral_1990-05-20
Name: name_dob, dtype: object

In [174]:
# Create column in mv_name_review in light of the manual review:

def id_review(row):
    
    if row['match_score'] >= 0.78 and row['pid'] not in remapping_ids.keys():
        decision = row['closest_match'] # This means that the match found by the sequence matcher is correct
    
    elif row['match_score'] >= 0.78 and row['pid'] in remapping_ids.keys():
        found_key = row['pid']
        temp_df = players_small[players_small['idn'] == remapping_ids[found_key]].reset_index()
        decision = temp_df.loc[0,'name_dob']
        
    elif row['match_score'] < 0.78 and row['pid'] in correct_ids_under_pt78:
        decision = row['closest_match'] # This means that the match found by the sequence matcher is correct
    
    elif row['match_score'] < 0.78 and row['pid'] in remapping_ids.keys():
        found_key = row['pid']
        temp_df = players_small[players_small['idn'] == remapping_ids[found_key]].reset_index()
        decision = temp_df.loc[0,'name_dob']
    
    else:
        decision = np.NaN
    
    return decision

mv_name_review['final_col'] = mv_name_review.apply(id_review, axis = 1)

In [175]:
mv_name_review[mv_name_review['final_col'].isnull()].shape

(40, 11)

In [176]:
# Insert the manual name check column into the main dataframe

def insert_name_review(row):
    if row['check'] != 0: # This checks if the automatic match was correct
        squawka_name_dob_val = row['name_dob']
        
    elif row['check'] == 0: # This falls back to the results of the manual check
        temp_df = mv_name_review[mv_name_review['pid'] == row['pid']].reset_index()
        squawka_name_dob_val = temp_df.loc[0,'final_col']
    
    return squawka_name_dob_val

mv_df['squawka_name_dob'] = mv_df.apply(insert_name_review, axis = 1)

In [177]:
mv_df[mv_df['squawka_name_dob'].notnull()].shape

(3137, 31)

In [178]:
# Drop null rows (i.e. where there is no squawka name match)

mv_df = mv_df.drop(mv_df[mv_df['squawka_name_dob'].isnull()].index)

mv_df.shape

(3137, 31)

In [179]:
players_small.head()

,idn,name,dob,name_dob,match_rate
0,817.0,Neil Taylor,1989-02-07,neil_taylor_1989-02-07,0.416667
1,810.0,Ashley Williams,1984-08-23,ashley_williams_1984-08-23,0.346154
2,382.0,Kyle Bartley,1991-05-22,kyle_bartley_1991-05-22,0.448980
3,375.0,Lukasz Fabianski,1985-04-18,lukasz_fabianski_1985-04-18,0.339623
4,3691.0,Ki Sung-yueng,1989-01-24,ki_sung-yueng_1989-01-24,0.320000


In [180]:
# In order to merge the squawka player id into the main dataframe, rename the squawka name_dob column to match

players_small.rename(columns={'name_dob':'squawka_name_dob'}, inplace=True)

In [181]:
players_small.head(5)

,idn,name,dob,squawka_name_dob,match_rate
0,817.0,Neil Taylor,1989-02-07,neil_taylor_1989-02-07,0.416667
1,810.0,Ashley Williams,1984-08-23,ashley_williams_1984-08-23,0.346154
2,382.0,Kyle Bartley,1991-05-22,kyle_bartley_1991-05-22,0.448980
3,375.0,Lukasz Fabianski,1985-04-18,lukasz_fabianski_1985-04-18,0.339623
4,3691.0,Ki Sung-yueng,1989-01-24,ki_sung-yueng_1989-01-24,0.320000


In [182]:
# Insert Squawka player id

mv_df = pd.merge(mv_df,players_small[['idn','squawka_name_dob']],how='left',on='squawka_name_dob')

In [184]:
mv_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3138 entries, 0 to 3137
Data columns (total 32 columns):
teamid              3138 non-null float64
number              3012 non-null float64
name                3138 non-null object
pid                 3138 non-null float64
main_pos            3138 non-null object
position            3138 non-null object
dob                 3138 non-null object
age                 3138 non-null float64
nat1                3138 non-null object
nat2                1020 non-null object
height              3122 non-null float64
foot                2963 non-null object
teamsince           3128 non-null object
prevteam            3128 non-null object
prevteamid          3073 non-null float64
transferfee         3138 non-null float64
contractuntil       3025 non-null object
marketval           3138 non-null float64
prevmarketval       3137 non-null float64
team                3138 non-null object
season              3138 non-null object
teamurl             313

## Save linked dataframe as CSV

In [185]:
# Convert dataframes to CSV

start_time=time.time()
start_time_stamp = str(start_time).replace(".","")
start_time_stamp

matched_file_name = "combined_table_" + start_time_stamp + ".csv"

mv_df.to_csv(matched_file_name, encoding="utf-8", index = False)

In [186]:
mv_df.columns

Index([u'teamid', u'number', u'name', u'pid', u'main_pos', u'position', u'dob',
       u'age', u'nat1', u'nat2', u'height', u'foot', u'teamsince', u'prevteam',
       u'prevteamid', u'transferfee', u'contractuntil', u'marketval',
       u'prevmarketval', u'team', u'season', u'teamurl', u'year',
       u'current_team', u'current_teamid', u'country', u'marketval_2016',
       u'country_2016', u'name_dob', u'check', u'squawka_name_dob', u'idn'],
      dtype='object')

In [187]:
mv_df['change'] = mv_df['marketval'] - mv_df['marketval_2016']

In [188]:
mv_df[mv_df['change'] < 0].shape

(1139, 33)

In [189]:
mv_df[mv_df['change'] > 0].shape

(1529, 33)

In [190]:
mv_df[mv_df['change'] == 0].shape

(470, 33)

In [835]:
470./3138

0.14977692797960485